# **검증 세트(validation set)**

기존에는 전체 데이터 중 80%를 훈련 세트 20%를 테스트 세트로 만들었다.

하지만 이제는 훈련 세트를 60%, 테스트 세트 20% 그리고 검증 세트 20%로 조정한다.

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)


(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# **교차 검증(cross validation)**

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.

그다음 점수를 평균하여 최종 검증 점수를 얻는다.

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0541122 , 0.01886797, 0.01313853, 0.01277065, 0.01479483]), 'score_time': array([0.00182557, 0.01678514, 0.00243187, 0.00188136, 0.00263143]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# **그리드 서치(Grid Search)**

하이퍼파리미터를 파이썬의 for문 대신 사이킷런에서 제공하는 툴로 매개변수를 바꿔가며 최적의 값을 찾아내는 방법

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


**과정 요약**


1.   먼저 탐색할 매개변수 지정

2.   그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾고 그리드 서치 객체에 저장

3.   그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련세트가 아니라)전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.



In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)
          }
#np.arange() -> 0.0001에서 0.001이 될 대까지 0.0001을 계속 더한 배열
#range() -> 5에서 20까지 1씩 증가하는 배 -> 총 15개
#min_samples에서는 2에서 100까지 10씩 증가하면서 10개의 값 생성
#교차 검증 횟수는 9 x 15 x 10 = 1,350개 -> 기존 5-폴드 교차 검증으로 모델의 수는 6,750개

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


**랜덤 서치(random search)**

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.
또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다.

이러한 상황에서 사용하는 것이 랜덤 서치이다.

램덤 서치에서는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

In [ ]:
from scipy.stats import uniform, randint
#randint는 정숫값, uniform은 실숫값을 뽑는다.
#각 클래스들은 모두 주어진 범위에서 고르게 값을 뽑는다(균등 분포에서 샘플링한다)

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

array([6, 0, 9, 0, 2, 3, 0, 6, 1, 6])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([113, 116, 105,  85, 101,  86,  84, 109, 104,  97]))

In [ ]:
ugen = uniform(0,1) # -> 0~1사이 실수 추출
ugen.rvs(10)

array([0.79703166, 0.56931369, 0.83017574, 0.06059649, 0.50479609,
       0.05219144, 0.93134279, 0.83428398, 0.50078167, 0.45053252])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a0797889990>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a0797889930>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a079788aef0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a079788bc70>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
